Stage 1: Data crawler. Massively parallel functions crawling data and storing it in Cloud Object Storage.  Obtain information from web pages or tweets and create a dataset of text data. Use FaaS backend in lithops to launch crawling process over serverless functions.

In [ ]:
import ipywidgets as widgets

analysis_folder_wg = widgets.Text(value='analysis1', placeholder='')
items = [widgets.Text(placeholder='String/hashtag #{}'.format(i)) for i in range(6)]

acc = widgets.Accordion(children=[analysis_folder_wg, widgets.HBox(items)])
acc.set_title(0, 'Folder name in storage to save data')
acc.set_title(1, 'Tags to crawl')
acc


In [ ]:
# You can add more hashtags if the don't fit the boxes manually here:
hashtags = []
for i in items:
    if i.value != '':
        hashtags.append(i.value)


analysis_folder_name = analysis_folder_wg.value

print('Hashtags:', hashtags)
print('Folder:',  analysis_folder)

# Save them on jupyter kernel so we can retrieve the hashtags on Stage 2 notebook
%store analysis_folder_name

In [ ]:
from custom_snscrape import TwitterSearchScraper

def scrapSearch(str_to_search, storage):

    tweets = ""
    counter = 0

    for tweet in TwitterSearchScraper(str_to_search).get_items():
        tweets = tweets + tweet.to_json() + '\n'

        counter = counter + 1
        if counter == 1000:
            break

    print(str_to_search)
    print('analysis', analysis_folder_name)
    # 'key' is only the sought hashtag, without the excluded ones
    storage.put_object(bucket=storage.bucket,
                        key=analysis_folder_name+'/'+str_to_search.split()[0],
                        body=tweets)

    return "OK"

In [ ]:
# Each query will include the current hashtag,
# and exclude the hashtags to the right of itself in the array.
# This way, we can parallelise the searches, while also ensuring that
# if a tweet contains multiple hashtags, one of the queries includes said tweet.
# 
# In this case, the queries will be:
# #covid -#sars -#coronavirus, #sars -#coronavirus, #coronavirus
#hashtags = ['#covid', '#sars', '#coronavirus']
queries = []

for i in range(len(hashtags)):
    queries.append(' -'.join(hashtags))
    hashtags.pop(0)

In [ ]:
import lithops

with lithops.FunctionExecutor() as fexec:
    all_invocations = fexec.map(scrapSearch, queries)
    print(fexec.get_result())